2025.04.25 最近tsinghua的pypi连接不上不知道为什么，用aliyun了。 

In [ ]:
! pip install numpy nibabel nilearn pandas h5py snirf -i https://mirrors.aliyun.com/pypi/simple

In [ ]:
# 测试通道信息
# 把转换好的通道MNI坐标做成一个nii文件以便于检查和调整

import numpy as np
import nibabel as nib
from nilearn.image import coord_transform
import pandas as pd

def is_mni_space(affine, tolerance=5):
    mni_affine = np.array([[  -2.,    0.,    0.,   90.],
                           [   0.,    2.,    0., -126.],
                           [   0.,    0.,    2.,  -72.],
                           [   0.,    0.,    0.,    1.]])
    return np.allclose(affine, mni_affine, atol=tolerance)

def mni_coords_to_nifti(coords, sphere_radius=5, binary=True, affine=None, template_shape=(91, 109, 91), output_filename='output.nii.gz'):
    if affine is None:
        affine = np.array([[  -2.,    0.,    0.,   90.],
                           [   0.,    2.,    0., -126.],
                           [   0.,    0.,    2.,  -72.],
                           [   0.,    0.,    0.,    1.]])

    if not is_mni_space(affine):
        raise ValueError("Input affine does not appear to be in MNI space.")

    img_data = np.zeros(template_shape, dtype=np.float32)

    for idx, coord in enumerate(coords, start=1):
        if len(coord) == 4:
            x, y, z, val = coord
        else:
            x, y, z = coord
            val = 1 if binary else idx

        i, j, k = np.round(nib.affines.apply_affine(np.linalg.inv(affine), [x, y, z])).astype(int)

        for a in range(-sphere_radius, sphere_radius + 1):
            for b in range(-sphere_radius, sphere_radius + 1):
                for c in range(-sphere_radius, sphere_radius + 1):
                    if a**2 + b**2 + c**2 <= sphere_radius**2:
                        ii, jj, kk = i + a, j + b, k + c
                        if (0 <= ii < template_shape[0] and
                            0 <= jj < template_shape[1] and
                            0 <= kk < template_shape[2]):
                            img_data[ii, jj, kk] = 1 if binary else val

    nifti_img = nib.Nifti1Image(img_data, affine)
    nib.save(nifti_img, output_filename)

if __name__ == "__main__":
    mni_coordinates = pd.read_csv('Layout2nifti.csv', header=0).values
    mni_coords_to_nifti(mni_coordinates, sphere_radius=2, binary=True, output_filename='binary_output.nii.gz')
    mni_coords_to_nifti(mni_coordinates, sphere_radius=2, binary=False, output_filename='weighted_output.nii.gz')


In [ ]:
# 写出header里面的通道信息以便核对layout 

import re

with open('20241019YAOLING_20241019_173242.TXT', 'r') as f:
    lines = [ln.rstrip('\n') for ln in f]

channel_pairs = []

header_lines = lines[:36]

in_text_info = False
pair_pattern = re.compile(r'\((\d+,\d+)\)')
for ln in header_lines:
    if ln.strip().startswith("[Text Info.]"):
        in_text_info = True
        continue
    if in_text_info and ln.strip().startswith("Time("):
        break
    if in_text_info:
        channel_pairs.extend([tuple(map(int, pair.split(','))) for pair in pair_pattern.findall(ln)])

for pair in channel_pairs:
    print(pair)

In [ ]:
# SampleData_Layout拆分为source detector channel

layout_csv = 'SampleData_Layout.csv'

import pandas as pd
import os

layout = pd.read_csv(layout_csv)

source = pd.DataFrame(columns=['Label', 'X', 'Y', 'Z'])
detector = pd.DataFrame(columns=['Label', 'X', 'Y', 'Z'])
channel = pd.DataFrame(columns=['Label', 'X', 'Y', 'Z'])
for index, row in layout.iterrows():
    if row['layout'].startswith('T'):
        source = pd.concat([source, pd.DataFrame([{'Label': row['layout'], 'X': row['x'], 'Y': row['y'], 'Z': row['z']}])], ignore_index=True)
    elif row['layout'].startswith('R'):
        detector = pd.concat([detector, pd.DataFrame([{'Label': row['layout'], 'X': row['x'], 'Y': row['y'], 'Z': row['z']}])], ignore_index=True)
    elif row['layout'].startswith('CH'):
        channel = pd.concat([channel, pd.DataFrame([{'Label': row['layout'], 'X': row['x'], 'Y': row['y'], 'Z': row['z']}])], ignore_index=True)
source.to_csv('source_coordinates.csv', index=False)
detector.to_csv('detector_coordinates.csv', index=False)
channel.to_csv('channel_coordinates.csv', index=False)



In [ ]:
# NIRS-SPM文件转换snirf
# 由gpt生成，仅保留了最少的必要信息
# metadata 中 SubjectAge SubjectSex SubjectHandedness 是固定的
# 需要从header获取metadata的时候就改一下

import re
import numpy as np
import h5py
import pandas as pd

def parse_header(header_path):
    meta = {}
    channel_pairs = []
    with open(header_path, 'r') as f:
        lines = [ln.rstrip('\n') for ln in f]
    if not lines:
        raise FileNotFoundError(f"Header file '{header_path}' is empty or not readable.")

    for ln in lines:
        if ln.startswith("Measured Date"):
            parts = ln.split(None, 2)
            if len(parts) >= 3:
                datetime_str = parts[2].strip()
                date_str, time_str = datetime_str.split(" ", 1) if " " in datetime_str else (datetime_str, "")
                meta['MeasurementDate'] = date_str
                meta['MeasurementTime'] = time_str
            break

    id_value = next((ln.split('\t')[1] for ln in lines if ln.startswith("ID")), "")
    name_value = next((ln.split('\t')[1] for ln in lines if ln.startswith("Name")), "")
    subj_id = id_value if len(id_value) >= len(name_value) else name_value
    meta['SubjectID'] = subj_id if subj_id else "Unknown"
    meta['SubjectAge'] = "18"
    meta['SubjectSex'] = "F"
    meta['SubjectHandedness'] = "Right"

    total_points = next((int(token) for ln in lines if ln.strip().startswith("Total Points")
                         for token in reversed(re.split(r'\s+', ln.strip())) if token.isdigit()), None)

    data_start_idx = None
    try:
        first_line = lines[0]
        match = re.search(r'\[Data Line\]\s*(\d+)', first_line)
        if match:
            data_start_idx = int(match.group(1)) - 1
        else:
            raise ValueError("[Data Line] indicator not found in first line.")
    except Exception:
        for i, ln in enumerate(lines):
            if ln.strip().startswith("Time("):
                data_start_idx = i + 1
                break
        if data_start_idx is None:
            raise ValueError("Could not locate the start of data in the header file.")

    header_lines = lines[:data_start_idx]
    data_lines = lines[data_start_idx:]

    in_text_info = False
    pair_pattern = re.compile(r'\((\d+,\d+)\)')
    for ln in header_lines:
        if ln.strip().startswith("[Text Info.]"):
            in_text_info = True
            continue
        if in_text_info and ln.strip().startswith("Time("):
            break
        if in_text_info:
            channel_pairs.extend([tuple(map(int, pair.split(','))) for pair in pair_pattern.findall(ln)])

    num_channels = 0
    if data_lines:
        col_count = len(data_lines[0].split())
        if col_count >= 5:
            num_meas_cols = col_count - 4
            num_channels = num_meas_cols // (3 if num_meas_cols % 3 == 0 else 2 if num_meas_cols % 2 == 0 else 1)

    if not channel_pairs or len(channel_pairs) != num_channels:
        channel_pairs = [(ch, ch) for ch in range(1, num_channels + 1)]

    wavelengths = []
    in_condition = False
    for ln in header_lines:
        if ln.strip().startswith("[Condition"):
            in_condition = True
            continue
        if in_condition and (ln.strip().startswith("[") or "Gain" in ln):
            break
        if in_condition:
            wavelengths += [float(num) for num in re.findall(r'\d+\.\d+', ln)]

    meta['Wavelengths'] = wavelengths

    times, data_matrix = [], []
    for ln in data_lines:
        parts = ln.split()
        if len(parts) < 5:
            continue
        try:
            times.append(float(parts[0]))
            data_matrix.append([float(val) if re.match(r'^-?\d+(\.\d+)?$', val) else np.nan for val in parts[4:]])
        except ValueError:
            continue

    times, data_matrix = np.array(times), np.array(data_matrix)

    if total_points is not None and total_points != len(times):
        print(f"Warning: Expected {total_points} data points, found {len(times)}.")

    return meta, channel_pairs, times, data_matrix

def load_coordinates(src_coords_csv, det_coords_csv):
    sources_df = pd.read_csv(src_coords_csv)
    detectors_df = pd.read_csv(det_coords_csv)

    sourcePos3D = sources_df[['X','Y','Z']].to_numpy()
    detectorPos3D = detectors_df[['X','Y','Z']].to_numpy()

    src_labels = sources_df['Label'].astype(str).tolist()
    det_labels = detectors_df['Label'].astype(str).tolist()

    return sourcePos3D, detectorPos3D, src_labels, det_labels

def write_snirf(output_path, meta, channel_pairs, times, data_matrix, sourcePos3D, detectorPos3D, src_map, det_map):
    with h5py.File(output_path, 'w') as f:
        f.create_dataset("formatVersion", data="1.1", dtype=h5py.string_dtype(encoding='utf-8'))
        nirs_grp = f.create_group("nirs")
        meta_grp = nirs_grp.create_group("metaDataTags")
        meta_grp.create_dataset("SubjectID", data=str(meta.get('SubjectID', "Unknown")), dtype=h5py.string_dtype(encoding='utf-8'))
        meta_grp.create_dataset("MeasurementDate", data=str(meta.get('MeasurementDate', "")), dtype=h5py.string_dtype(encoding='utf-8'))
        meta_grp.create_dataset("MeasurementTime", data=str(meta.get('MeasurementTime', "")), dtype=h5py.string_dtype(encoding='utf-8'))
        meta_grp.create_dataset("LengthUnit", data="mm", dtype=h5py.string_dtype(encoding='utf-8'))
        meta_grp.create_dataset("TimeUnit", data="s", dtype=h5py.string_dtype(encoding='utf-8'))
        meta_grp.create_dataset("FrequencyUnit", data="Hz", dtype=h5py.string_dtype(encoding='utf-8'))
        for field in ("SubjectAge", "SubjectSex", "SubjectHandedness"):
            if field in meta:
                meta_grp.create_dataset(field, data=str(meta[field]), dtype=h5py.string_dtype(encoding='utf-8'))
        data_grp = nirs_grp.create_group("data1")
        data_grp.create_dataset("dataTimeSeries", data=data_matrix)
        data_grp.create_dataset("time", data=times)
        num_channels = len(channel_pairs)
        num_measurements = data_matrix.shape[1] if data_matrix.size > 0 else 0
        meas_per_channel = num_measurements // num_channels if num_channels > 0 else 0
        if meas_per_channel * num_channels != num_measurements:
            meas_per_channel = 1
        ml_index = 1
        for (orig_src, orig_det) in channel_pairs:
            for m in range(meas_per_channel):
                ml_grp = data_grp.create_group(f"measurementList{ml_index}")
                ml_grp.create_dataset("sourceIndex", data=np.int32(src_map.get(orig_src, 0)))
                ml_grp.create_dataset("detectorIndex", data=np.int32(det_map.get(orig_det, 0)))
                wavelength_list = meta.get('Wavelengths', [])
                wl_idx = 1
                if wavelength_list:
                    wl_idx = m+1 if meas_per_channel <= len(wavelength_list) else 1
                ml_grp.create_dataset("wavelengthIndex", data=np.int32(wl_idx))
                ml_grp.create_dataset("dataType", data=np.int32(99999))
                data_label = None
                if meas_per_channel == 3:
                    data_labels = ["HbO", "HbR", "HbT"]
                    data_label = data_labels[m] if m < len(data_labels) else "ProcessedData"
                else:
                    data_label = "ProcessedData"
                ml_grp.create_dataset("dataTypeLabel", data=data_label, dtype=h5py.string_dtype(encoding='utf-8'))
                data_unit = "M" if data_label in {"HbO", "HbR", "HbT"} else "a.u."
                ml_grp.create_dataset("dataUnit", data=data_unit, dtype=h5py.string_dtype(encoding='utf-8'))
                ml_grp.create_dataset("dataTypeIndex", data=np.int32(m+1))
                ml_index += 1
        probe_grp = nirs_grp.create_group("probe")
        probe_grp.create_dataset("wavelengths", data=np.array(meta.get('Wavelengths', []), dtype=np.float64))
        probe_grp.create_dataset("sourcePos3D", data=sourcePos3D)
        probe_grp.create_dataset("detectorPos3D", data=detectorPos3D)
        src_labels = [f"S{orig}" for orig in sorted(src_map.keys(), key=lambda x: src_map[x])]
        det_labels = [f"D{orig}" for orig in sorted(det_map.keys(), key=lambda x: det_map[x])]
        probe_grp.create_dataset("sourceLabels", data=np.array(src_labels, dtype=h5py.string_dtype(encoding='utf-8')))
        probe_grp.create_dataset("detectorLabels", data=np.array(det_labels, dtype=h5py.string_dtype(encoding='utf-8')))
    print(f"SNIRF file successfully saved as '{output_path}'")

if __name__ == "__main__":
    nirsspm = 'SampleData.TXT'
    src_csv = 'source_coordinates.csv'
    det_csv = 'detector_coordinates.csv'
    meta, channel_pairs, times, data_matrix = parse_header(nirsspm)
    sourcePos3D, detectorPos3D, src_labels, det_labels = load_coordinates(src_csv, det_csv)
    src_map = {int(label[1:]): i + 1 for i, label in enumerate(src_labels)}
    det_map = {int(label[1:]): i + 1 for i, label in enumerate(det_labels)}
    if nirsspm.lower().endswith(('.txt')):
        out_path = nirsspm.rsplit('.', 1)[0] + ".snirf"
    else:
        out_path = nirsspm + ".snirf"
    write_snirf(out_path, meta, channel_pairs, times, data_matrix, sourcePos3D, detectorPos3D, src_map, det_map)


In [ ]:
# layout加入到snirf
# 用法是上面的代码没加入layout时使用
# 如果加了就不用了 

import h5py
import numpy as np
import pandas as pd

def write_mni_coordinates(snirf_file, src_coords_csv, det_coords_csv):
    sources_df = pd.read_csv(src_coords_csv)
    detectors_df = pd.read_csv(det_coords_csv)

    sourcePos3D = sources_df[['X','Y','Z']].to_numpy()
    detectorPos3D = detectors_df[['X','Y','Z']].to_numpy()

    src_labels = sources_df['Label'].astype(str).tolist()
    det_labels = detectors_df['Label'].astype(str).tolist()

    with h5py.File(snirf_file, 'a') as f:
        probe_grp = f.require_group("/nirs/probe")
        probe_grp.create_dataset("sourcePos3D", data=sourcePos3D)
        probe_grp.create_dataset("detectorPos3D", data=detectorPos3D)
        probe_grp.create_dataset("sourceLabels", data=np.array(src_labels, dtype=h5py.string_dtype()))
        probe_grp.create_dataset("detectorLabels", data=np.array(det_labels, dtype=h5py.string_dtype()))
        probe_grp.attrs['coordinateSystem'] = "MNI"
        probe_grp.attrs['coordinateUnits'] = "mm"

    print(f"MNI coordinates successfully written to '{snirf_file}'.")

if __name__ == '__main__':
    snirf_path = "SampleData.snirf"
    source_csv = "source_coordinates.csv"
    detector_csv = "detector_coordinates.csv"
    write_mni_coordinates(snirf_path, source_csv, detector_csv)


In [ ]:
from snirf import validateSnirf
result = validateSnirf(r'SampleData.snirf')
result.display(severity=3)

# 备用方案：使用Shimadzu2nirs先转换到nirs文件，再通过homer3转换到snirf

下载[Shimadzu2nirs](https://www.nitrc.org/projects/shimadzu2nirs)

原脚本（Sahar Jahani， 04-14-2017）问题如下：

- 在44行，str2num没有把37行的数据split，直接执行获得NaN
- 在68行，还是没有split，所以通道从string变成了错误的list，每个element不能执行regexprep
- 在89行，没有清空变量，当文件有多个时（时间点不同）会报错

或者下载我修改好的文件。

或者使用下面的脚本（gpt基于Shimadzu2nirs原版修改的文件）。

相比于上面的write_snirf，Shimadzu2nirs进行了MBL的转换到optical density，homer3转换的是v1.0的snirf，validateSnirf有warning，不知道具体是什么。

Shimadzu2nirs+Homer3
```
Found 1031 OK      (hidden)
Found 266 INFO    (hidden)
Found 504 WARNING (hidden)
Found 0 FATAL  
File is VALID
```

而write_snirf只需要0.4s，txt直接到snirf没有warning，数据是raw intensity，snirf版本v1.1，不依靠pysnirf2（这个package也很怪）。

```
Found 896 OK      (hidden)
Found 899 INFO    (hidden)
Found 0 WARNING (hidden)
Found 0 FATAL  
File is VALID
```


In [ ]:
import os
import re
import numpy as np
from scipy.io import loadmat, savemat

# Module Description (mirroring the MATLAB script):
"""
Conversion of Shimadzu data (.txt) to HOMER2 readable format (.nirs).

This module provides a function Shimadzu2nirs() that reads all Shimadzu fNIRS 
data files (.txt) in the current directory, along with a corresponding .SD 
file (MATLAB format containing probe layout), and converts them into .nirs 
files usable in the HOMER2 NIRS processing package.

Output .nirs files will contain the following variables:
- d:     intensity data (time points x channels)
- aux:   auxiliary data (time points x 1) – filled with zeros
- s:     stimulus markers (time points x 1) – 1 at stimulus onsets (all stimuli combined)
- t:     time vector (time points x 1)
- tIncMan: manually included time points (time points x 1) – here all ones (no manual exclusion)
- SD:    probe layout structure (as loaded from the .SD file)
"""

def get_extinctions(wavelengths):
    """
    Get extinction coefficients for HbO and Hb at specified wavelengths.

    Returns the extinction (specific absorption) coefficients for HbO and Hb 
    (in units of 1/(M*cm) base-e) for each input wavelength (nm). Also returns 
    coefficients for water, lipid, and cytochrome aa3 if needed (5 columns total).
    """
    # Data for molar extinction coefficients (base 10) of HbO and Hb from 250nm to 1000nm.
    # Compiled by Scott Prahl (see Gratzer & Kollias data). Converted to base-e later.
    # Each tuple: (wavelength_nm, ext_HbO, ext_Hb) in [cm^-1/(M)] base-10 units.
    extinction_data = [
        (250.0, 106112.0, 112736.0), (252.0, 105552.0, 112736.0), (254.0, 107660.0, 112736.0),
        (256.0, 109788.0, 113824.0), (258.0, 112944.0, 115040.0), (260.0, 116376.0, 116296.0),
        (262.0, 120188.0, 117564.0), (264.0, 124412.0, 118876.0), (266.0, 128696.0, 120208.0),
        (268.0, 133064.0, 121544.0), (270.0, 136068.0, 122880.0), (272.0, 137232.0, 123096.0),
        (274.0, 138408.0, 121952.0), (276.0, 137424.0, 120808.0), (278.0, 135820.0, 119840.0),
        (280.0, 131936.0, 118872.0), (282.0, 127720.0, 117628.0), (284.0, 122280.0, 114820.0),
        (286.0, 116508.0, 112008.0), (288.0, 108484.0, 107140.0), (290.0, 104752.0, 98364.0),
        (292.0, 98936.0, 91636.0),  (294.0, 88136.0, 85820.0),  (296.0, 79316.0, 77100.0),
        (298.0, 70884.0, 69444.0),  (300.0, 65972.0, 64440.0),  (302.0, 63208.0, 61300.0),
        (304.0, 61952.0, 58828.0),  (306.0, 62352.0, 56908.0),  (308.0, 62856.0, 57620.0),
        (310.0, 63352.0, 59156.0),  (312.0, 65972.0, 62248.0),  (314.0, 69016.0, 65344.0),
        (322.0, 82256.0, 78284.0),  (324.0, 85972.0, 82060.0),  (326.0, 89796.0, 85592.0),
        (328.0, 93768.0, 88516.0),  (330.0, 97512.0, 90856.0),  (332.0, 100964.0, 93192.0),
        (334.0, 103504.0, 95532.0), (336.0, 104968.0, 99792.0), (338.0, 106452.0, 104476.0),
        (340.0, 107884.0, 108472.0), (342.0, 109060.0, 110996.0), (344.0, 110092.0, 113524.0),
        (346.0, 109032.0, 116052.0), (348.0, 107984.0, 118752.0), (350.0, 106576.0, 122092.0),
        (352.0, 105040.0, 125436.0), (354.0, 103696.0, 128776.0), (356.0, 101568.0, 132120.0),
        (358.0, 97828.0, 133632.0), (360.0, 94744.0, 134940.0), (362.0, 92248.0, 136044.0),
        (364.0, 89836.0, 136972.0), (366.0, 88484.0, 137900.0), (368.0, 87512.0, 138856.0),
        (370.0, 88176.0, 139968.0), (372.0, 91592.0, 141084.0), (374.0, 95140.0, 142196.0),
        (376.0, 98936.0, 143312.0), (378.0, 103432.0, 144424.0), (380.0, 109564.0, 145232.0),
        (382.0, 116968.0, 145232.0), (384.0, 125420.0, 148668.0), (386.0, 135132.0, 153908.0),
        (388.0, 148100.0, 159544.0), (390.0, 167748.0, 167780.0), (392.0, 189740.0, 180004.0),
        (394.0, 212060.0, 191540.0), (396.0, 231612.0, 202124.0), (398.0, 248404.0, 212712.0),
        (400.0, 266232.0, 223296.0), (402.0, 284224.0, 236188.0), (404.0, 308716.0, 253368.0),
        (406.0, 354208.0, 270548.0), (408.0, 422320.0, 287356.0), (410.0, 466840.0, 303956.0),
        (412.0, 500200.0, 321344.0), (414.0, 524280.0, 342596.0), (416.0, 521880.0, 363848.0),
        (418.0, 515520.0, 385680.0), (420.0, 480360.0, 407560.0), (422.0, 431880.0, 429880.0),
        (424.0, 376236.0, 461200.0), (426.0, 326032.0, 481840.0), (428.0, 283112.0, 500840.0),
        (430.0, 246072.0, 528600.0), (432.0, 214120.0, 552160.0), (434.0, 165332.0, 552160.0),
        (436.0, 132820.0, 547040.0), (438.0, 119140.0, 501560.0), (440.0, 102580.0, 413280.0),
        (442.0, 92780.0, 363240.0),  (444.0, 81444.0, 282724.0),  (446.0, 76324.0, 237224.0),
        (448.0, 67044.0, 173320.0),  (450.0, 62816.0, 103292.0),  (452.0, 58864.0, 62640.0),
        (454.0, 53552.0, 36170.0),   (456.0, 49496.0, 30698.8),   (458.0, 47496.0, 25886.4),
        (460.0, 44480.0, 23388.8),   (462.0, 41320.0, 20891.2),   (464.0, 39807.2, 19260.8),
        (466.0, 37073.2, 18142.4),   (468.0, 34870.8, 17025.6),   (470.0, 33209.2, 16156.4),
        (472.0, 31620.0, 15310.0),   (474.0, 30113.6, 15048.4),   (476.0, 28850.8, 14792.8),
        (478.0, 27718.0, 14657.2),   (480.0, 26629.2, 14550.0),   (482.0, 25701.6, 14881.2),
        (484.0, 25180.4, 15212.4),   (486.0, 24669.6, 15543.6),   (488.0, 24174.8, 15898.0),
        (490.0, 23684.4, 16684.0),   (492.0, 23086.8, 17469.6),   (494.0, 22457.6, 18255.6),
        (496.0, 21850.4, 19041.2),   (498.0, 21260.0, 19891.2),   (500.0, 20932.8, 20862.0),
        (502.0, 20596.4, 21832.8),   (504.0, 20418.0, 22803.6),   (506.0, 19946.0, 23774.4),
        (508.0, 19996.0, 24745.2),   (510.0, 20035.2, 25773.6),   (512.0, 20150.4, 26936.8),
        (514.0, 20429.2, 28100.0),   (516.0, 21001.6, 29263.2),   (518.0, 22509.6, 30426.4),
        (520.0, 24202.4, 31589.6),   (522.0, 26450.4, 32851.2),   (524.0, 29269.2, 34397.6),
        (526.0, 32496.4, 35944.0),   (528.0, 35990.0, 37490.0),   (530.0, 39956.8, 39036.4),
        (532.0, 43876.0, 40584.0),   (534.0, 46924.0, 42088.0),   (536.0, 49752.0, 43592.0),
        (538.0, 51712.0, 45092.0),   (540.0, 53236.0, 46592.0),   (542.0, 53292.0, 48148.0),
        (544.0, 52096.0, 49708.0),   (546.0, 49868.0, 51268.0),   (548.0, 46660.0, 52496.0),
        (550.0, 43016.0, 53412.0),   (552.0, 39675.2, 54080.0),   (554.0, 36815.2, 54520.0),
        (556.0, 34476.8, 54540.0),   (558.0, 33456.0, 54164.0),   (560.0, 32613.2, 53788.0),
        (562.0, 32620.0, 52276.0),   (564.0, 33915.6, 50572.0),   (566.0, 36495.2, 48828.0),
        (568.0, 40172.0, 46948.0),   (570.0, 44496.0, 45072.0),   (572.0, 49172.0, 43340.0),
        (574.0, 53308.0, 41716.0),   (576.0, 55540.0, 40092.0),   (578.0, 54728.0, 38467.6),
        (580.0, 50104.0, 37020.0),   (582.0, 43304.0, 35676.4),   (584.0, 34639.6, 34332.8),
        (586.0, 26600.4, 32851.6),   (588.0, 19763.2, 31075.2),   (590.0, 14400.8, 28324.4),
        (592.0, 10468.4, 25470.0),   (594.0, 7678.8, 22574.8),    (596.0, 5683.6, 19800.0),
        (598.0, 4504.4, 17058.4),    (600.0, 3200.0, 14677.2),    (602.0, 2664.0, 13622.4),
        (604.0, 2128.0, 12567.6),    (606.0, 1789.2, 11513.2),    (608.0, 1647.6, 10477.6),
        (610.0, 1506.0, 9443.6),     (612.0, 1364.4, 8591.2),     (614.0, 1222.8, 7762.0),
        (616.0, 1110.0, 7344.8),     (618.0, 1026.0, 6927.2),     (620.0, 942.0, 6509.6),
        (622.0, 858.0, 6193.2),      (624.0, 774.0, 5906.8),      (626.0, 707.6, 5620.0),
        (628.0, 658.8, 5366.8),      (630.0, 610.0, 5148.8),      (632.0, 561.2, 4930.8),
        (634.0, 512.4, 4730.8),      (636.0, 478.8, 4602.4),      (638.0, 460.4, 4473.6),
        (640.0, 442.0, 4345.2),      (642.0, 423.6, 4216.8),      (644.0, 405.2, 4088.4),
        (646.0, 390.4, 3965.08),     (648.0, 379.2, 3857.6),      (650.0, 506.0, 3743.0),
        (652.0, 488.0, 3677.0),      (654.0, 474.0, 3612.0),      (656.0, 464.0, 3548.0),
        (658.0, 454.3, 3491.3),      (660.0, 445.0, 3442.0),      (662.0, 438.3, 3364.7),
        (664.0, 433.8, 3292.8),      (666.0, 431.3, 3226.3),      (668.0, 429.0, 3133.0),
        (670.0, 427.0, 3013.0),      (672.0, 426.5, 2946.0),      (674.0, 426.0, 2879.0),
        (676.0, 424.0, 2821.7),      (678.0, 423.0, 2732.3),      (680.0, 423.0, 2610.8),
        (682.0, 422.0, 2497.3),      (684.0, 420.0, 2392.0),      (686.0, 418.0, 2292.7),
        (688.0, 416.5, 2209.3),      (690.0, 415.5, 2141.8),      (692.0, 415.0, 2068.7),
        (694.0, 415.0, 1990.0),      (696.0, 415.5, 1938.5),      (698.0, 416.0, 1887.0),
        (700.0, 419.3, 1827.7),      (702.0, 422.5, 1778.5),      (704.0, 425.5, 1739.5),
        (706.0, 429.7, 1695.7),      (708.0, 435.0, 1647.0),      (710.0, 441.0, 1601.7),
        (712.0, 446.5, 1562.5),      (714.0, 451.5, 1529.5),      (716.0, 458.0, 1492.0),
        (718.0, 466.0, 1450.0),      (720.0, 472.7, 1411.3),      (722.0, 479.5, 1380.0),
        (724.0, 486.5, 1356.0),      (726.0, 494.3, 1331.7),      (728.0, 503.0, 1307.0),
        (730.0, 510.0, 1296.5),      (732.0, 517.0, 1286.0),      (734.0, 521.0, 1286.0),
        (736.0, 530.7, 1293.0),      (738.0, 546.0, 1307.0),      (740.0, 553.5, 1328.0),
        (742.0, 561.0, 1349.0),      (744.0, 571.0, 1384.3),      (746.0, 581.3, 1431.3),
        (748.0, 592.0, 1490.0),      (750.0, 600.0, 1532.0),      (752.0, 608.0, 1574.0),
        (754.0, 618.7, 1620.7),      (756.0, 629.7, 1655.3),      (758.0, 641.0, 1678.0),
        (760.0, 645.5, 1669.0),      (762.0, 650.0, 1660.0),      (764.0, 666.7, 1613.3),
        (766.0, 681.0, 1555.0),      (768.0, 693.0, 1485.0),      (770.0, 701.5, 1425.0),
        (772.0, 710.0, 1365.0),      (774.0, 722.0, 1288.3),      (776.0, 733.7, 1216.3),
        (778.0, 745.0, 1149.0),      (780.0, 754.0, 1107.5),      (782.0, 763.0, 1066.0),
        (784.0, 775.0, 1021.3),      (786.0, 787.0, 972.0),       (788.0, 799.0, 918.0),
        (790.0, 808.0, 913.0),       (792.0, 817.0, 908.0),       (794.0, 829.0, 887.3),
        (796.0, 840.7, 868.7),       (798.0, 852.0, 852.0),       (800.0, 863.3, 838.7),
        (802.0, 873.3, 828.0),       (804.0, 881.8, 820.0),       (806.0, 891.7, 812.0),
        (808.0, 903.0, 804.0),       (810.0, 914.3, 798.7),       (812.0, 924.7, 793.7),
        (814.0, 934.0, 789.0),       (816.0, 943.0, 787.0),       (818.0, 952.0, 785.0),
        (820.0, 962.7, 783.0),       (822.0, 973.0, 781.0),       (824.0, 983.0, 779.0),
        (826.0, 990.5, 778.5),       (828.0, 998.0, 778.0),       (830.0, 1008.0, 778.0),
        (832.0, 1018.0, 777.7),      (834.0, 1028.0, 777.0),      (836.0, 1038.0, 777.0),
        (838.0, 1047.7, 777.0),      (840.0, 1057.0, 777.0),      (842.0, 1063.5, 777.5),
        (844.0, 1070.0, 778.0),      (846.0, 1079.3, 779.3),      (848.0, 1088.3, 780.3),
        (850.0, 1097.0, 781.0),      (852.0, 1105.7, 783.0),      (854.0, 1113.0, 785.0),
        (856.0, 1119.0, 787.0),      (858.0, 1126.0, 789.3),      (860.0, 1134.0, 792.0),
        (862.0, 1142.0, 795.3),      (864.0, 1149.7, 799.0),      (866.0, 1157.0, 803.0),
        (868.0, 1163.7, 807.7),      (870.0, 1170.3, 812.3),      (872.0, 1177.0, 817.0),
        (874.0, 1182.0, 820.5),      (876.0, 1187.0, 824.0),      (878.0, 1193.0, 830.0),
        (880.0, 1198.7, 835.7),      (882.0, 1204.0, 841.0),      (884.0, 1209.3, 847.0),
        (886.0, 1214.3, 852.7),      (888.0, 1219.0, 858.0),      (890.0, 1223.7, 863.3),
        (892.0, 1227.5, 867.8),      (894.0, 1230.5, 871.3),      (896.0, 1234.0, 875.3),
        (898.0, 1238.0, 880.0),      (900.0, 1241.3, 883.3),      (902.0, 1202.0, 765.04),
        (904.0, 1206.0, 767.44),     (906.0, 1209.2, 769.8),      (908.0, 1211.6, 772.16),
        (910.0, 1214.0, 774.56),     (912.0, 1216.4, 776.92),     (914.0, 1218.8, 778.4),
        (916.0, 1220.8, 778.04),     (918.0, 1222.4, 777.72),     (920.0, 1224.0, 777.36),
        (922.0, 1225.6, 777.04),     (924.0, 1227.2, 776.64),     (926.0, 1226.8, 772.36),
        (928.0, 1224.4, 768.08),     (930.0, 1222.0, 763.84),     (932.0, 1219.6, 752.28),
        (934.0, 1217.2, 737.56),     (936.0, 1215.6, 722.88),     (938.0, 1214.8, 708.16),
        (940.0, 1214.0, 693.44),     (942.0, 1213.2, 678.72),     (944.0, 1212.4, 660.52),
        (946.0, 1210.4, 641.08),     (948.0, 1207.2, 621.64),     (950.0, 1204.0, 602.24),
        (952.0, 1200.8, 583.4),      (954.0, 1197.6, 568.92),     (956.0, 1194.0, 554.48),
        (958.0, 1190.0, 540.04),     (960.0, 1186.0, 525.56),     (962.0, 1182.0, 511.12),
        (964.0, 1178.0, 495.36),     (966.0, 1173.2, 473.32),     (968.0, 1167.6, 451.32),
        (970.0, 1162.0, 429.32),     (972.0, 1156.4, 415.28),     (974.0, 1150.8, 402.28),
        (976.0, 1144.0, 389.288),    (978.0, 1136.0, 374.944),    (980.0, 1128.0, 359.656),
        (982.0, 1120.0, 344.372),    (984.0, 1112.0, 329.084),    (986.0, 1102.4, 313.796),
        (988.0, 1091.2, 298.508),    (990.0, 1080.0, 283.22),     (992.0, 1068.8, 267.932),
        (994.0, 1057.6, 252.648),    (996.0, 1046.4, 237.36),     (998.0, 1035.2, 222.072),
        (1000.0, 1024.0, 206.784)
    ]
    # Convert the data to numpy arrays for interpolation
    ext_data_np = np.array(extinction_data)
    known_wavelengths = ext_data_np[:, 0]
    ext_hbo_base10 = ext_data_np[:, 1]
    ext_hb_base10 = ext_data_np[:, 2]
    # Ensure input is array for vectorized operations
    wavelengths = np.array(wavelengths, dtype=float).flatten()
    num_lambda = wavelengths.size
    # Prepare output array: 5 columns (HbO, Hb, H2O, lipid, aa3), initialize zeros
    exs = np.zeros((num_lambda, 5))
    # We handle only 250-1000 nm range for interpolation
    # Interpolate HbO and Hb (base-10 ext. coeff.) and convert to base-e
    mask = (wavelengths >= 250) & (wavelengths <= 1000)
    if np.any(mask):
        exs[mask, 0] = np.interp(wavelengths[mask], known_wavelengths, ext_hbo_base10) * 2.303  # HbO
        exs[mask, 1] = np.interp(wavelengths[mask], known_wavelengths, ext_hb_base10) * 2.303   # Hb
    # (Optional: we could similarly fill H2O, lipid, aa3 columns if needed using known spectra,
    # but for this conversion we only require HbO and Hb columns.)
    return exs

def hmrConc2OD(dc, SD, ppf):
    """
    Convert concentration data to optical density (OD) changes.

    Parameters:
        dc : numpy array of shape (time_points, 3, channels)
             Concentration changes (HbO, HbR, HbT) for each source-detector pair.
        SD : SD structure (as loaded from .SD file) containing probe info.
        ppf: partial pathlength factors for each wavelength (list or array of length nWav).

    Returns:
        dod : numpy array of shape (time_points, Nwavelengths*channels)
              Optical density change for each measurement (each source-detector at each wavelength).
    """
    # Number of wavelengths in the measurement (length of SD.Lambda)
    nWav = int(np.size(SD['Lambda']))  # SD['Lambda'] could be a 1xN array in MATLAB struct
    # Ensure ppf length matches number of wavelengths
    if len(ppf) != nWav:
        raise ValueError("The length of ppf must match the number of wavelengths in SD.Lambda.")
    # Flatten ppf to array for use
    ppf = np.array(ppf).flatten()
    # Measurement list (ml) from SD. 
    # SD['MeasList'] in the loaded structure might be an array of shape (Nmeas, 4).
    ml = SD['MeasList']
    # If ml is stored as a 2D array within a 1x1 object, extract it
    if ml.ndim == 2:
        MeasList = ml
    else:
        MeasList = ml[0,0]
    MeasList = MeasList.astype(int)  # ensure integer type for indexing
    nTpts = dc.shape[0]  # number of time points
    # Get extinction coefficients for the wavelengths (base-e, per cm)
    lambdas = SD['Lambda'].astype(float).flatten()
    e = get_extinctions(lambdas)
    # Use only the first two columns (HbO and Hb) and convert to per mm by dividing by 10
    e = e[:, 0:2] / 10.0  # shape: (nWav x 2), in 1/(M*mm)
    # Initialize dod array: time_points x (number of measurements)
    nMeas = MeasList.shape[0]
    dod = np.zeros((nTpts, nMeas))
    # Find indices in MeasList for first wavelength measurements
    # (MeasList columns: source, detector, ?, wavelength index)
    wave1_indices = np.where(MeasList[:, 3] == 1)[0]  # 0-indexed indices where wavelength index = 1
    # Loop over each source-detector pair (for the first wavelength entries)
    for idx, idx1 in enumerate(wave1_indices):
        # idx1 is the MeasList index (0-indexed in Python) for a measurement with wavelength 1
        src = MeasList[idx1, 0]
        det = MeasList[idx1, 1]
        # Find the corresponding measurement indices for the other wavelengths (>1) with same source and det
        other_idx = np.where((MeasList[:, 0] == src) & (MeasList[:, 1] == det) & (MeasList[:, 3] > 1))[0]
        # Combine idx1 and other wavelength indices
        all_idx = np.concatenate(([idx1], other_idx))
        # Compute distance between source and detector positions (euclidean norm in mm)
        # Note: SD['SrcPos'] and SD['DetPos'] could be stored as separate arrays in the structure.
        src_positions = SD['SrcPos']
        det_positions = SD['DetPos']
        if src_positions.ndim > 1:
            src_pos = src_positions[src-1, :]  # subtract 1 for MATLAB-to-Python index
            det_pos = det_positions[det-1, :]
        else:
            # If loaded as object, access first element
            src_pos = src_positions[0,0][src-1, :]
            det_pos = det_positions[0,0][det-1, :]
        rho = np.linalg.norm(src_pos - det_pos)
        # Compute change in OD for this source-detector pair across all wavelengths
        # Use only HbO and HbR concentrations (dc[:,0:2,channel_index])
        # The channel_index corresponds to this source-detector pair in dc's 3rd dimension.
        # We assume dc's third index ordering corresponds to the order of unique source-detector pairs.
        # The idx (0-based loop index) should correspond to channel index in dc for this pair.
        conc_O = dc[:, 0, idx]  # HbO for this channel (time x 1)
        conc_R = dc[:, 1, idx]  # HbR for this channel
        # Form a 2 x time array for conc to multiply with extinction matrix
        conc_mat = np.stack((conc_O, conc_R))  # shape (2, nTpts)
        # Compute OD changes: (e (nWav x 2) * conc_mat (2 x nTpts))' gives (nTpts x nWav)
        od_changes = (e.dot(conc_mat)).T  # shape (nTpts, nWav)
        # Apply pathlength factor: multiply each wavelength's OD by (rho * ppf[wavelength_index])
        # ppf is length nWav.
        od_changes *= (rho * ppf)
        # Place the computed OD changes into the dod matrix columns for the corresponding MeasList indices
        # Note: all_idx contains the indices of MeasList for this source-detector pair (all wavelengths)
        dod[:, all_idx] = od_changes
    return dod

def Shimadzu2nirs():
    """
    Convert all Shimadzu .txt files in the current directory to .nirs files.
    
    Reads each .txt file (Shimadzu data) and the .SD file (probe layout), 
    then outputs a .nirs file with intensity data and associated variables.
    """
    # Find all .txt data files in current directory
    txt_files = [f for f in os.listdir(os.getcwd()) if f.lower().endswith('.txt')]
    # If no text files found, do nothing
    for txt_file in txt_files:
        # Load the SD file (there should be one .SD file in the directory)
        sd_files = [f for f in os.listdir(os.getcwd()) if f.lower().endswith('.sd')]
        if len(sd_files) == 0:
            raise FileNotFoundError("No .SD file found in current directory.")
        SD_file = sd_files[0]
        # Load SD structure from .SD file using scipy.io.loadmat
        SD_data = loadmat(SD_file)
        # Get the SD structure (it might be nested in a 1x1 cell, depending on MATLAB version)
        SD = SD_data.get('SD')
        if SD is None:
            raise KeyError("SD structure not found in .SD file.")
        # If SD is a 1x1 object array (struct), extract the object
        if isinstance(SD, np.ndarray) and SD.dtype == 'O':
            SD = SD[0, 0]
        # Open and read the text file
        with open(txt_file, 'r') as f:
            lines = f.read().splitlines()
        # Extract channel mapping information from line 33 (index 32 if 0-indexed)
        if len(lines) < 33:
            raise ValueError(f"File {txt_file} has insufficient lines to extract channel info.")
        channel_info_line = lines[32]
        # Remove parentheses and commas to isolate numbers
        cleaned = re.sub(r'[(),]', ' ', channel_info_line)
        numbers = [int(num) for num in cleaned.split() if num.isdigit()]
        # Reshape into pairs of [Source, Detector]
        if len(numbers) % 2 != 0:
            raise ValueError("Channel info line does not contain an even number of entries.")
        num_channels = len(numbers) // 2
        channel_pairs = [(numbers[i], numbers[i+1]) for i in range(0, len(numbers), 2)]
        # Read numeric data from line 37 onward (index 36 and beyond)
        # We use numpy to load the data for speed and simplicity
        data_array = np.loadtxt(txt_file, skiprows=36)
        # Separate time, stim markers, and concentration data
        t = data_array[:, 0]             # time vector
        stim_markers = data_array[:, 2]  # stimulus marker (combined)
        # Create tIncMan as all ones (keep all time points)
        tIncMan = np.ones_like(t)
        # Create aux as zeros (same length as time)
        aux = np.zeros_like(t)
        # Extract concentration data (HbO, HbR, HbT):
        # Concentration columns start at index 4 (5th column in file) and repeat every 3 columns
        conc_data = data_array[:, 4:]  # from 5th column to end
        # Determine number of channels from concentration data (should match channel_pairs length)
        # conc_data has shape (time_points, 3 * num_channels)
        calc_num_channels = conc_data.shape[1] // 3
        if calc_num_channels != num_channels:
            num_channels = calc_num_channels  # override if different (for safety)
        # Reshape concentration into dc (time_points x 3 x num_channels)
        # We know conc_data ordering is [HbO1, HbR1, HbT1, HbO2, HbR2, HbT2, ..., HbON, HbRN, HbTN]
        time_points = conc_data.shape[0]
        dc = conc_data.reshape(time_points, num_channels, 3)
        # Swap axes to shape (time_points, 3, num_channels)
        dc = np.transpose(dc, (0, 2, 1))
        # Convert concentrations (HbO/HbR) to optical density changes
        ppf = [6.0] * int(np.size(SD['Lambda']))  # partial pathlength factor (6 for each wavelength)
        dod = hmrConc2OD(dc, SD, ppf)
        # Convert OD changes to raw intensity (d) by exponentiating (-OD * 1e-4)
        d = np.exp(-dod * 1e-4)
        # Reorder intensity data in d to match the SD.MeasList order
        # Build a list of [Src, Det, WavelengthIndex] for each measurement in the original text order
        # Original text order for channels is given by channel_pairs (src,det) and each has nWav wavelengths.
        nWav = int(np.size(SD['Lambda']))
        list_measurements = []
        for wave_idx in range(1, nWav+1):
            for (src, det) in channel_pairs:
                list_measurements.append((src, det, wave_idx))
        # Create a mapping from (src, det, wave) to column index in original d
        original_order_map = {val: idx for idx, val in enumerate(list_measurements)}
        # Get MeasList from SD (ensuring it's a numpy array of int)
        ml = SD['MeasList']
        if isinstance(ml, np.ndarray):
            if ml.dtype == 'O':  # if it's object, get the numeric array inside
                ml = ml[0, 0]
        MeasList = np.array(ml, dtype=int)
        num_meas = MeasList.shape[0]
        reordered_d = np.zeros((time_points, num_meas))
        for i in range(num_meas):
            src_i = MeasList[i, 0]
            det_i = MeasList[i, 1]
            wav_i = MeasList[i, 3]
            key = (src_i, det_i, wav_i)
            if key in original_order_map:
                orig_col = original_order_map[key]
                reordered_d[:, i] = d[:, orig_col]
            else:
                raise KeyError(f"Measurement (Src={src_i}, Det={det_i}, Wave={wav_i}) not found in original data order.")
        d = reordered_d
        # Prepare variables to save. Shape them to 2D arrays (column vectors where appropriate)
        t = t.reshape(-1, 1)
        s = stim_markers.reshape(-1, 1)
        tIncMan = tIncMan.reshape(-1, 1)
        aux = aux.reshape(-1, 1)
        # Prepare output file name (same base name as input .txt, with .nirs extension)
        base_name = os.path.splitext(txt_file)[0]
        out_file = base_name + '.nirs'
        # Save .nirs file (MATLAB .mat format with .nirs extension)
        savemat(out_file, {'d': d, 'aux': aux, 's': s, 't': t, 'tIncMan': tIncMan, 'SD': SD})
